In [1]:
from convert_ast import from_dict,to_dict,file_to_dict
from localization import Localization
from pycparser import parse_file, c_parser, c_generator, c_ast

In [71]:
file_dict = file_to_dict("src.c")

In [72]:
file_dict['ext'][8]

{'_nodetype': 'FuncDef',
 'coord': 'src.c:18:1',
 'decl': {'_nodetype': 'Decl',
  'name': 'p11_dict_new',
  'quals': [],
  'storage': [],
  'funcspec': [],
  'coord': 'src.c:18:1',
  'type': {'_nodetype': 'FuncDecl',
   'coord': 'src.c:18:1',
   'type': {'_nodetype': 'PtrDecl',
    'quals': [],
    'coord': 'src.c:17:10',
    'type': {'_nodetype': 'TypeDecl',
     'declname': 'p11_dict_new',
     'quals': [],
     'coord': 'src.c:18:1',
     'type': {'_nodetype': 'IdentifierType',
      'names': ['p11_dict'],
      'coord': 'src.c:17:1'}}},
   'args': None},
  'init': None,
  'bitsize': None},
 'body': {'_nodetype': 'Compound',
  'coord': 'src.c:18:1',
  'block_items': [{'_nodetype': 'Decl',
    'name': 'dict',
    'quals': [],
    'storage': [],
    'funcspec': [],
    'coord': 'src.c:19:14',
    'type': {'_nodetype': 'PtrDecl',
     'quals': [],
     'coord': 'src.c:19:14',
     'type': {'_nodetype': 'TypeDecl',
      'declname': 'dict',
      'quals': [],
      'coord': 'src.c:19:15

In [73]:
def localization(ast_dict,line):
    s=ast_dict
    if type(s)==dict:
        coord=int(s['coord'].split(":")[1])
        if coord==line:
            return s
        for v in s.values():
            if not v:
                continue
            elif type(v)==dict:
                if int(v['coord'].split(":")[1])>line:
                    return localization(tmp,line)
            elif type(v)==list:
                if int(v[0]['coord'].split(":")[1])>line:
                    return localization(tmp,line)
            else:
                continue
            tmp=v
        return localization(tmp,line)

    elif type(s)==list:
        if len(s)==1:
            return localization(s[0],line)
        for i in range(1,len(s)):
            coord=int(s[i]['coord'].split(":")[1])
            if coord>line:
                return localization(s[i-1],line)
        return localization(s[-1],line)

In [74]:
localization(file_dict['ext'],23)

{'_nodetype': 'Assignment',
 'op': '=',
 'coord': 'src.c:23:9',
 'lvalue': {'_nodetype': 'StructRef',
  'type': '.',
  'coord': 'src.c:23:9',
  'name': {'_nodetype': 'StructRef',
   'type': '->',
   'coord': 'src.c:23:9',
   'name': {'_nodetype': 'ID', 'name': 'dict', 'coord': 'src.c:23:9'},
   'field': {'_nodetype': 'ID', 'name': 'buckets', 'coord': 'src.c:23:15'}},
  'field': {'_nodetype': 'ID', 'name': 'aa', 'coord': 'src.c:23:23'}},
 'rvalue': {'_nodetype': 'Cast',
  'coord': 'src.c:23:28',
  'to_type': {'_nodetype': 'Typename',
   'name': None,
   'quals': [],
   'coord': 'src.c:0:1',
   'type': {'_nodetype': 'PtrDecl',
    'quals': [],
    'coord': 'src.c:23:41',
    'type': {'_nodetype': 'PtrDecl',
     'quals': [],
     'coord': 'src.c:23:40',
     'type': {'_nodetype': 'TypeDecl',
      'declname': None,
      'quals': [],
      'coord': None,
      'type': {'_nodetype': 'IdentifierType',
       'names': ['dictbucket'],
       'coord': 'src.c:23:29'}}}}},
  'expr': {'_nodetype

In [66]:
localization(file_dict['ext'],20)

{'_nodetype': 'Assignment',
 'op': '=',
 'coord': 'src.c:20:5',
 'lvalue': {'_nodetype': 'ID', 'name': 'dict', 'coord': 'src.c:20:5'},
 'rvalue': {'_nodetype': 'FuncCall',
  'coord': 'src.c:20:12',
  'name': {'_nodetype': 'ID', 'name': 'malloc', 'coord': 'src.c:20:12'},
  'args': {'_nodetype': 'ExprList',
   'coord': 'src.c:20:20',
   'exprs': [{'_nodetype': 'UnaryOp',
     'op': 'sizeof',
     'coord': 'src.c:20:20',
     'expr': {'_nodetype': 'Typename',
      'name': None,
      'quals': [],
      'coord': 'src.c:0:1',
      'type': {'_nodetype': 'TypeDecl',
       'declname': None,
       'quals': [],
       'coord': None,
       'type': {'_nodetype': 'IdentifierType',
        'names': ['p11_dict'],
        'coord': 'src.c:20:28'}}}}]}}}

In [81]:
def get_heap_object(file_dict,line):
    s=localization(file_dict['ext'],line)
    if(s['_nodetype']=='Decl'):
        name=s['name']
        del_type=''
        s=s['type']
        while True:
            if s["_nodetype"]=="PtrDecl":
                del_type+=" *"
                s=s["type"]
            elif s["_nodetype"]=="TypeDecl":
                s=s["type"]
            elif s["_nodetype"]=="IdentifierType":
                del_type=s["names"][0]+del_type
                break
            elif s["_nodetype"]=="Struct":
                del_type="struct "+s["name"]+del_type
                break
    elif(s['_nodetype']=='Assignment'):
        s_name=s['lvalue']
        name=''
        while True:
            if s_name['_nodetype']=='ID':
                name=s_name['name']+name
                break
            elif s_name['_nodetype']=='StructRef':
                name=s_name['type']+s_name['field']['name']+name
                s_name=s_name['name']
        del_type=''
        s_type=s['rvalue']
        if s['_nodetype']=='FuncCall':
            
            
        elif s['_nodetype']=='Cast':
    else:
        print("(heap_object_type/get_heap_object) Can not get heap object, nodetype is : "+s['_nodetype'])

    return name,del_type

In [ ]:
def find_decl_location():
    pass

In [82]:
get_heap_object(file_dict,23)

('dict->buckets.aa', '')

In [7]:
file_ast = from_dict(file_dict)
generator = c_generator.CGenerator()

print(generator.visit(file_ast))

typedef struct _p11_dict
{
  struct p11_dictbucket **buckets;
} p11_dict;
typedef struct p11_dictbucket
{
  struct p11_dictbucket *next;
} dictbucket;
int num = 0;
p11_dict *p1_1;
dictbucket **p1_2;
dictbucket **p2_1;
int c1_1;
int c2_1;
p11_dict *p11_dict_new()
{
  p11_dict *dict;
  dict = malloc(sizeof(p11_dict));
  if (num == 1)
    p1_1 = dict;

  if (dict)
  {
    dict->buckets = (dictbucket **) malloc(sizeof(dictbucket *));
    if (num == 1)
      p1_2 = dict->buckets;

    if (num == 2)
      p2_1 = dict->buckets;

    if (!dict->buckets)
    {
      free(dict);
      return NULL;
    }

  }

  return dict;
}

p11_dict *p11_constant_reverse(int nick)
{
  p11_dict *lookups;
  if (num == 1)
    c1_1 = nick;

  lookups = p11_dict_new();
  if (nick)
  {
    return NULL;
  }
  else
  {
    return lookups;
  }

}

p11_dict *func(int c)
{
  num = 2;
  if (num == 2)
    c2_1 = c;

  p11_dict *dict;
  dict = p11_dict_new();
  if (c == 0)
  {
    free(dict);
    return NULL;
  }

  return